# Tech Challenge 3: Arquitetura ML e Aprendizado

## Introducão

*Problema a ser resolvido*

Identificar a correlação entre dados de usuário (idade, gênero, região, ocupação), preferência por filmes e sua avaliação.
A partir desse modelo, será criada uma API que poderá receber dois inputs diferentes:

1. Dados pertencentes a uma pessoa (idade, gênero, ...) e partir deles e do modelo treinado ser capaz de sugerir filmes em que poderiam potencialmente ter interesse.

2. Dados relacionados a um filmes (tempo de duração, gêneros, ano de lançamento) e partir deles e do modelo treinado ser capaz de indicar o grupo de maior interesse, para que um time de marketing, por exemplo saiba pra onde direcionar as campanhas

## Setup

*Conjunto de Dados*

Usaremos o grupo de dados do Movielens para esse caso de estudo (https://files.grouplens.org/datasets/movielens/)

In [ ]:
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install xgboost
%pip install requests
%pip install boto3
%pip install python-dotenv

In [41]:
import pandas as pd
import zipfile
import re
import numpy as np
import requests
import os
import boto3
from io import StringIO
from dotenv import load_dotenv
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Perceptron
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

## Load dos dados

In [90]:
url = "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
file_path = "ml-1m.zip"
response = requests.get(url)
with open(file_path, 'wb') as file:
        file.write(response.content)

In [91]:
with zipfile.ZipFile('ml-1m.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [92]:
os.remove(file_path)

In [ ]:
user_data = pd.read_csv('dataset/ml-1m/users.dat',sep='::',header=None,names=['userid','gender','age','occupation','zipcode'],encoding='iso-8859-2')
movie_data = pd.read_csv('dataset/ml-1m/movies.dat',sep='::',header=None,names=['movieid','title','genres'],encoding='iso-8859-2')
ratings_data = pd.read_csv('dataset/ml-1m/ratings.dat',sep='::',header=None,usecols=[0,1,2],names=['userid','movieid','rating'],encoding='iso-8859-2')


## Pre-processamento

### Estrutura de cada dataset

In [ ]:
user_data.head()

In [ ]:
movie_data.head()

In [ ]:
ratings_data.head()

### Analisa os dados

Ao analisar os dados, podemos perceber que possuímos um ratio desbalanceado entre usuários homens (M) e usuárias mulheres (F).

In [ ]:
user_data['gender'].value_counts().plot(kind='pie', autopct='%1.0f%%')

Antes de prosseguirmos uma outra informação que é interessante de analisarmos é a distribuição etária de nossos usuários. Vale a pena resaltar que a coluda idade (age) já está "normalizada", obedecendo a seguinte proporção:
*  1:  Mais novos que 18 anos
* 18:  18-24 anos
* 25:  25-34 anos
* 35:  35-44 anos
* 45:  45-49 anos
* 50:  50-55 anos
* 56:  56+ anos

In [ ]:
user_data.plot.hist(column=["age"], by="gender", figsize=(10, 8))

### Normaliza os dados

Ao analisarmos os dados, mais precisamente a coluna *genres* do dataset de Filmes, é possível perceber que temos um problema de 'Multi-Label' nos nossos dados, onde um filme pode pertencer a 1 ou N gêneros de filmes. Para contornarmos essa situação, iremos converter essa coluna para uma lista de string:

In [106]:
movie_data["release_year"] = [int(re.search(r'\((\d{4})\)', x).group(1)) for x in movie_data["title"]]
movie_data['genres'] = movie_data["genres"].str.split("|")

In [ ]:
movie_data.head()

Em seguida, utilizaremos o **MultiLabelBinarizer** para transformar essa coluna em uma matriz binária

In [108]:
mlb = MultiLabelBinarizer()
encoded_data = mlb.fit_transform(movie_data['genres'])
dados_encodados = pd.DataFrame(encoded_data, columns=mlb.classes_)

In [ ]:
dados_encodados.head()

E finalmente concatenaremos esse novo dataframe ao "original", podendo assim excluir a coluna '*genres*' de nossos dados.

In [ ]:
movie_data = pd.concat([movie_data,dados_encodados], axis=1)
movie_data = movie_data.drop(['genres'], axis=1)
movie_data.head()

Partindo de um princípio parecido, a coluna Gender do dataset de usuário que está dividido em M e F, pode ser alterada um LabelEncoder para facilitar nosso treinamento de modelos.

In [111]:
label_encoder = LabelEncoder()
user_data['gender'] = label_encoder.fit_transform(user_data['gender'])

In [ ]:
user_data['gender'].head()

### Merge e limpeza dos dados

Antes de começarmos a testar quais modelos podem nos ajudar a solucionar o nosso problema, ou até mesmo antes de padronizarmos/tratarmos os nossos dados, iremos criar um quarto Dataset, que irá conter o todo do cenário com o qual pretendemos trabalhar

In [ ]:
dados_completos = pd.merge(user_data, ratings_data, on='userid')
dados_completos = pd.merge(dados_completos, movie_data, on='movieid')
dados_completos = dados_completos.sample(n=5000, random_state=23)
dados_completos.head()

In [ ]:
dados_modelo = dados_completos.drop(['movieid','userid','title','zipcode'], axis=1)
dados_modelo.head()


In [ ]:
dados_modelo.info()

In [ ]:
dados_modelo.isna().sum()

### Separa dados de treino de dados de teste

Agora, podemos começar a nossa jornada de validação e testes de alguma modelos, como temos a coluna rating, que funciona como a nota do especialista, podemos começar com um aprendizado supervisionado. Começaremos com modelos de regressão para tal. Contudo, antes de comerçamos com os treinamentos, iremos separar nossos dados em dados de treino e dados de teste, utilizando o train_test_split do scikitlearn

Primeiramente separamos os nossos dados X (features) e Y (target).

In [24]:
x = dados_modelo.drop(columns=['rating'], axis=1)
y = dados_modelo['rating']

Agora separamos a parcela de treino (80%) e teste (20%)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23)

## Utils

Já deixaremos aqui, uma função para a avaliação do Modelo e uma função para busca dos melhores hiper parâmetros

In [27]:
def avalia_modelo(y_test,y_pred, tipo_modelo):
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print(f'{tipo_modelo} - MSE: {mse} RMSE: {rmse} MAPE: {mape}')

def procura_melhores_parametros(model, parametros, treino_x, treino_y):
    grid_search = GridSearchCV(estimator=model,
                           param_grid=parametros,
                           cv=5,
                           scoring='neg_mean_absolute_percentage_error')

    # busca a melhor combinação de parametros, baseado nos dados de teste
    grid_search.fit(treino_x, treino_y)

    # retorna o melhor modelo
    melhor_modelo = grid_search.best_estimator_
    return melhor_modelo  


## Algoritmos

***REGRESSÂO LINEAR***

Começaremos os nossos testes, utilizando o LinearRegression do scikit-learn

In [ ]:
model = procura_melhores_parametros(LinearRegression(), {}, x_train, y_train)
lr_pred =  model.predict(x_test)

avalia_modelo(y_test, lr_pred, 'Linear')

***KNN Regressor***

Agora utilizaremos o KNN Regressor, contudo para esse modelo utilizaremos a função previamente criada para chegar no melhor número de "vizinhos"

In [29]:

parametros = {
    'n_neighbors': [2, 5, 8, 10],
    'metric':['euclidean']
}

model = procura_melhores_parametros(KNeighborsRegressor(), parametros, x_train, y_train)
knnregressor_pred = model.predict(x_test)

***Support Vector Machine: Linear***

O SVM é uma ferramenta robusta, poderosa e versátil. Indicada para problemas mais complexos e alcançar alta precisão, vamos ver como se sai em nosso caso de estudo. Mais uma vez obtendo o melhor modelo a partir do gridsearch

In [30]:
parametros = {
    'C': [5, 25, 60, 100]
}

model = procura_melhores_parametros(SVR(kernel='linear'), parametros, x_train, y_train)
svr_pred =  model.predict(x_test)

***Árvore de Decisão***

A árvore de decisão contrói uma estrutura hierárquica de regras que dividem em subconjuntos baseados nas características, facilitando a identificação

In [31]:
parametros = {}

model = procura_melhores_parametros(DecisionTreeClassifier(), parametros, x_train, y_train)
decision_tree_pred =  model.predict(x_test)

***Redes Neurais Artificiais (RNA)***

As redes neurais artificiais são modelos computacionais que imitam o funcionamento do cérebro humano para resolver problemas complexos

In [32]:
parametros = {}

model = procura_melhores_parametros(Perceptron(), parametros, x_train, y_train)
perceptron_pred =  model.predict(x_test)

***Multilayer Perceptron (MLP)***

O multilayer percepton é um modelo de rene neural artificial com múltiplas camadas

In [ ]:
parametros = {}

model = procura_melhores_parametros(MLPClassifier(), parametros, x_train, y_train)
mlp_pred =  model.predict(x_test)

***XGBoost***

Finalizaremos nossos testes com diferente tipos de estimadores, fazendo uso do XGBoost. É um dos mais utilizados pelo mercado e academia, funciona atráves da construção sequencial de N árvores de decisão mais fracas, onde cada árvore aprende com o erro da anterior.

In [34]:
parametros = {
    "n_estimators": [20, 50, 100],
    "max_depth": [3, 6, 8],
    "learning_rate": [0.1]
}

model = procura_melhores_parametros(XGBRegressor(), parametros, x_train, y_train)

xgboost_pred = model.predict(x_test)

## Resultado

Agora, uma vez que temos os estimadores treinados, mas definir através do MAPE. Qual modelo performa melhor em nosso cenário de estudo:

In [ ]:
avalia_modelo(y_test,lr_pred,"LinearRegression")
avalia_modelo(y_test,knnregressor_pred,"KNN Regressor")
avalia_modelo(y_test,svr_pred,"SVM Linear")
avalia_modelo(y_test,xgboost_pred,"XGBoost")
avalia_modelo(y_test,decision_tree_pred,"DecisionTree")
avalia_modelo(y_test,perceptron_pred,"Perceptron")
avalia_modelo(y_test,mlp_pred,"MLP")

Agora que sabemos que o melhor estimador pro nosso cenário de estudo, foi o {INSERIR AQUI MENOR O DE MENOR MAPE} cujo o MAPE foi de {INSERIR VALOR DO MAPE}. Lembrando que poderiamos tentar otimizar ainda mais, utilizando uma gama maior de hyper parâmetros, contudo poderiamos cair em cenário onde ficariamos extremamente especializados com nossos dados de teste (Overfitting).

Podemos finalmente exportar nosso modelo e avançar para a próxima etapa que é construir uma API que consiga predizer para um usuário de acordo com diferentes dados de entrada.

## Upload dataset S3

Cria instancia do Boto3

In [37]:
load_dotenv()

bucket_name = 'techchallengegp53'

s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN')
)

try:
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    bucket_connected = True
except Exception as e:
    bucket_connected = False

Verificar se existe conexão com o Bucket. Se não existir, savamos local no FakeBucket

In [ ]:
def save_dataframe_as_dat_locally(dataframe, file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w', encoding='iso-8859-2') as f:
         for index, row in dataframe.iterrows():
            f.write('::'.join(map(str, row.values)) + '\n')
    print(f"Arquivo {file_path} salvo com sucesso.")

bucket_folder = 'fakeBucket'
user_data_file_path = os.path.join(bucket_folder, 'user_data.dat')
movie_data_file_path = os.path.join(bucket_folder, 'movie_data.dat')

if not bucket_connected:
    save_dataframe_as_dat_locally(user_data, user_data_file_path)
    save_dataframe_as_dat_locally(movie_data, movie_data_file_path)

Salvando movie_data em um bucket S3

In [39]:
if bucket_connected:

    movie_data_buffer = StringIO()

    for index, row in movie_data.iterrows():
        movie_data_buffer.write('::'.join(map(str, row.values)) + '\n')
    s3_client.put_object(Bucket=bucket_name, Key="movie_data.dat", Body=movie_data_buffer.getvalue())

## Cleanup

In [ ]:
import shutil

pastas_remocao = ['dataset', 'fakeBucket']

for pasta in pastas_remocao:
    if (os.path.exists(pasta)):
        shutil.rmtree(pasta)